**Spectral analysis**

Change these parameters! Set the folder of EEG files.

*NB! The path has to be in 'Data\Clean\' folder relative to this notebook.*

In [ ]:
"""
VARIABLES TO CHANGE
exp_folder: A relative folder location of the experiment (e.g. r"Eyes Closed\Baseline")
exp_condition: A short code of the experiment which will be used for exporting (e.g. r"EC_00")
b_names: A list of bands names
b_freqs: A list of bands' frequency ranges
"""
exp_folder = r"Eyes Closed\Baseline"
exp_condition = r"EC_00"
b_names = ['Delta','Theta','Alpha','Low-Beta','High-Beta']
#b_freqs = [[0.5,3],[3.1,6.5],[6.6,12],[12.1,18],[18.1,30]]
b_freqs = [[1,3.9],[4,7.9],[8,12],[12.1,18],[18.1,30]]

Just run this script! It conducts spectral analysis on clean EEG files and save the results regionally and channel-by-channel in Excel format.
It does the following step-by-step:
1. Read raw FIF file (cleaned EEG file)
2. Calculate power spectrum density for the current file
3. Calculate bandpower for each frequency band of interest (e.g., delta, theta, alpha, low-beta, high-beta)
4. Split the signal to half for each band and plot the comparison if the median absolute deviation is over 2 across the signal
5. Plot all the frequency bands topographically
6. Attempt to create a folder and save the bandpowers regionally and channel-by-channel to Excel files

In [ ]:
from Functions.fn_spectralanalysis import *
from Functions.fn_basic import read_files, array_to_df, df_channels_to_regions

import os, mne
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

clean_folder = r"Data\Clean"
dir_inprogress = os.path.join(clean_folder,exp_folder)
file_dirs, subject_names = read_files(dir_inprogress,"_clean-epo.fif")
df_faa = pd.DataFrame(columns=['Pre-frontal','Mid-frontal','Lateral-frontal'])

# Go through all the files (subjects) in the folder
for i in range(len(file_dirs)):
    # Read the clean data from the disk
    epochs = mne.read_epochs(fname='{}\{}_clean-epo.fif'.format(dir_inprogress,subject_names[i]),verbose=False)
    
    # Calculate Welch's power spectrum density (FFT)
    [psds,freqs] = calculate_psd(epochs,subject_names[i],epo_duration=2,fminmax=[1,100],window='hamming')

    # Total PSD of chosen frequency range
    psd_ch_allbands = bandpower_per_channel(psds,freqs,[b_freqs[0][0],b_freqs[-1][-1]],'All bands',subject_names[i],epochs)
    
    # Find power for all bands and add to dataframe including the previous subjects
    vmin = float('inf')
    vmax = 0
    for j in range(len(b_names)):
        if i == 0:
            globals()["df_psd_"+b_names[j]] = pd.DataFrame()
            globals()["df_rel_psd_"+b_names[j]] = pd.DataFrame()
        
        # Devide the PSD to frequency band bins and calculate absolute and relative bandpowers
        globals()["psd_ch_"+b_names[j]] = bandpower_per_channel(psds,freqs,b_freqs[j],b_names[j],subject_names[i],epochs)
        globals()["rel_psd_ch_"+b_names[j]] = globals()["psd_ch_"+b_names[j]] / psd_ch_allbands

        # Convert the array to dataframe and concatenate it to dataframe including the previous subjects
        globals()["temp_df_psd_"+b_names[j]] = array_to_df(subject_names[i],epochs,globals()["psd_ch_"+b_names[j]])
        globals()["df_psd_"+b_names[j]] = pd.concat([globals()["df_psd_"+b_names[j]],globals()["temp_df_psd_"+b_names[j]]])
        globals()["temp_df_rel_psd_"+b_names[j]] = array_to_df(subject_names[i],epochs,globals()["rel_psd_ch_"+b_names[j]])
        globals()["df_rel_psd_"+b_names[j]] = pd.concat([globals()["df_rel_psd_"+b_names[j]],globals()["temp_df_rel_psd_"+b_names[j]]])

        # Save the minimum and maximum PSD values as an integer for later colorbar use
        vmin = min([vmin,min(globals()["psd_ch_"+b_names[j]])])
        vmax = max([vmax,max(globals()["psd_ch_"+b_names[j]])])

    # Plot topomaps for all bands
    sns.set_style("white",{'font.family': ['sans-serif']})
    fig,axs = plt.subplots(nrows=1,ncols=len(b_names),figsize=(10,3))
    fig.suptitle("Frequency topomaps ({})".format(subject_names[i]))
    for topo in range(len(b_names)):
        im,_ = mne.viz.plot_topomap(globals()["psd_ch_"+b_names[topo]],epochs.info,axes=axs[topo],vmin=vmin,vmax=vmax,show=False)
        axs[topo].set_title(b_names[topo]+'\n'+str(b_freqs[topo]))
    cbar_ax = fig.add_axes([0.95,0.35,0.04,0.4])
    clb = fig.colorbar(im, cax=cbar_ax)
    clb.ax.set_ylabel('uV\u00b2/Hz')

df_faa['Pre-frontal'] = (df_psd_Alpha['Fp1'] - df_psd_Alpha['Fp2'])/(df_psd_Alpha['Fp1'] + df_psd_Alpha['Fp2'])*100
df_faa['Mid-frontal'] = (df_psd_Alpha['F3'] - df_psd_Alpha['F4'])/(df_psd_Alpha['F3'] + df_psd_Alpha['F4'])*100
df_faa['Lateral-frontal'] = (df_psd_Alpha['F7'] - df_psd_Alpha['F8'])/(df_psd_Alpha['F7'] + df_psd_Alpha['F8'])*100
# df_faa['Pre-frontal'] = np.log(df_psd_Alpha['Fp1'])-np.log(df_psd_Alpha['Fp2'])
# df_faa['Mid-frontal'] = np.log(df_psd_Alpha['F3'])-np.log(df_psd_Alpha['F4'])
# df_faa['Lateral-frontal'] = np.log(df_psd_Alpha['F7'])-np.log(df_psd_Alpha['F8'])

# Try to pre-create folders for PSD results
try:
    os.makedirs(os.path.join(r"Results\Absolute PSD\channels",exp_folder))
except FileExistsError:
    pass
try:
    os.makedirs(os.path.join(r"Results\Absolute PSD\regions",exp_folder))
except FileExistsError:
    pass
try:
    os.makedirs(os.path.join(r"Results\Relative PSD\channels",exp_folder))
except FileExistsError:
    pass
try:
    os.makedirs(os.path.join(r"Results\Relative PSD\regions",exp_folder))
except FileExistsError:
    pass

df_faa.to_excel(r"Results\Absolute PSD\{}_frontal_asymmetry.xlsx".format(exp_condition))

for band in b_names:
    # Save the PSD values for each channel for each band in Excel format
    globals()["df_psd_"+band].to_excel(r"Results\Absolute PSD\channels\{}\{}_psd_{}.xlsx".format(exp_folder,exp_condition,band))
    globals()["df_rel_psd_"+band].to_excel(r"Results\Relative PSD\channels\{}\{}_rel_psd_{}.xlsx".format(exp_folder,exp_condition,band))

    # Find regional band powers and save them to Excel as well
    globals()["df_psd_reg_"+band] = df_channels_to_regions(globals()["df_psd_"+band])
    globals()["df_psd_reg_"+band].to_excel(r"Results\Absolute PSD\regions\{}\{}_psd_{}.xlsx".format(exp_folder,exp_condition,band))
    globals()["df_rel_psd_reg_"+band] = df_channels_to_regions(globals()["df_rel_psd_"+band])
    globals()["df_rel_psd_reg_"+band].to_excel(r"Results\Relative PSD\regions\{}\{}_rel_psd_{}.xlsx".format(exp_folder,exp_condition,band))